# Create an adjacency matrix that is time consistent

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import json
from pandas.io.json import json_normalize
from scipy import sparse

In [2]:
# Load our dataframes
participations = pd.read_pickle('../../data/raw/participation_data')
projects = pd.read_pickle("../../data/raw/project_data")

In [3]:
# Examine Participations
participations.head()

,authenticated,duration,extra,index_,magnitude,origin,profile,profile_origin,profile_referrer,profile_utm_campaign,profile_utm_content,profile_utm_medium,profile_utm_source,profile_utm_term,project,repetitions,type,when,where
0,True,0,,1,1,Unspecified,c3174748ab29f73d8c6226d0c2171aeb,,,,,,,,25.0,1,Participated,2016-07-22 14:07:43,NaN
1,True,0,,2,1,Unspecified,c3174748ab29f73d8c6226d0c2171aeb,,,,,,,,25.0,1,Participated,2016-08-26 17:43:54,NaN
2,True,0,,3,1,Unspecified,59d1c4ccba844b6f4722d2967c531441,,,,,,,,25.0,1,Participated,2016-08-26 17:46:32,NaN
3,True,0,,4,1,Unspecified,59d1c4ccba844b6f4722d2967c531441,,,,,,,,25.0,1,Participated,2016-08-26 17:48:18,NaN
4,True,0,,5,1,Unspecified,59d1c4ccba844b6f4722d2967c531441,,,,,,,,25.0,1,Participated,2016-08-28 23:57:25,NaN


In [4]:
# Find entries where the project_id is nan and remove these elements
nan_projects = participations[participations['project'].isnull()]
print(len(participations))
print(len(nan_projects))
participations = participations[~participations['project'].isnull()]
print(len(participations))

1474748
213179
1261569


In [5]:
# Remove elements where the type of participation isnt positive
# "Removed a bookmark", "Removed from dashboard"
participations = participations[participations['type'] != "Removed a bookmark"]
participations = participations[participations['type'] != "Removed from dashboard"]

In [6]:
# Sort by 'when' column
participations = participations.sort_values('when')

In [7]:
# Get all unique profiles in the participation dataset
profiles = participations['profile'].unique()

In [8]:
profile_projects = participations.groupby('profile')['project'].agg([list, len]).reset_index()
profile_projects = profile_projects.rename(index=str, columns={"list": "projects", "len":"num_projects"})    

In [9]:
profile_projects.head()

,profile,projects,num_projects
0,000073c3675ea9a1d0fe0ee3ca57e2bf,"[413.0, 413.0]",2.0
1,0002b85e757486c6d80ed6f73f465eaa,[16864.0],1.0
2,0003a41bbdb3371df4c1829913f17537,[19794.0],1.0
3,000436aaa487e461e6e16e02ab3e89eb,[659.0],1.0
4,000476d8680db78d75b3b9edefc4a6d2,[1510.0],1.0


In [10]:
len(profile_projects)

73131

In [11]:
profile_projects.to_pickle('../../data/processed/profile_projects_time_consistent')